In [5]:
import sys
import Utils
import pyDOE
import pandas as pd
import numpy as np
import scipy.stats.distributions as dist
from sklearn import preprocessing
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
from sklearn.preprocessing import MinMaxScaler

## Helper Functions

In [6]:
''' Elastic Net Regression '''
def elastic_net(train_data,test_data,rat):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values, test_data.values])
    regr = ElasticNet(alpha= 10202.350099442046 ,random_state=0 , l1_ratio=rat, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    return regr,pred

""" Generating Polynomial Features i.e., Function Basis """
def quadratic_polynomial (df):
    return pd.DataFrame(PolynomialFeatures(degree=2).fit_transform(df))

""" Quadratic Regression with Elastic Net Penalty"""
def polynomial(tr, te):
    f_original = train['Y']
    true = test['Y']
    temp1 = quadratic_polynomial (tr.iloc[:,:-1])
    temp2 = quadratic_polynomial (te.iloc[:,:-1])
    temp1 ['Y'] = f_original
    model_eln , pred_eln = elastic_net(temp1,temp2)
    return model_eln , pred_eln

## Load Training and Test Data Set initially Generated

In [7]:
path = "Training_Data_Sets\\train_2_1000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv(path[:-42]+str('Test_Data_Sets\\test_2_200Samples.csv')).iloc[:,1:]
for i in range(1,51):
    train ['X'+str(i)+'**2'] = train['X'+str(i)]**2
for i in range(1,51):
    for j in range(i+1,51):
        train ['X'+str(i)+'X'+str(j)] = train['X'+str(i)] * train ['X'+str(j)]
f_original = train['Y']
del train['Y']
train['Y'] = f_original
for i in range(1,51):
    test ['X'+str(i)+'**2'] = test['X'+str(i)]**2
for i in range(1,51):
    for j in range(i+1,51):
        test ['X'+str(i)+'X'+str(j)] = test['X'+str(i)] * test ['X'+str(j)]
true = test['Y']
del test['Y']
test['Y'] = true

## ELN

In [8]:
scaler =  MinMaxScaler().fit(np.r_[train.iloc[:,:-1].values, test.iloc[:,:-1].values])
regr = ElasticNetCV(cv=5,random_state=0)
regr.fit(scaler.transform(np.array(train.iloc[:,:-1])) , np.array(train.iloc[:,-1]))
print (regr.alpha_)

10202.350099442046


## ELN

In [9]:
Hyper_Parameters = np.linspace(0,1,100)
mean_abs_error = np.zeros(Hyper_Parameters.shape[0])
for i in range(Hyper_Parameters.shape[0]):
    model,pred_k_m = elastic_net(train,test.iloc[:,:-1],Hyper_Parameters[i])
    mean_abs_error[i] = np.mean((abs(true-pred_k_m) / abs(true) ) * 100)

C:\Users\nle450026\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [10]:
print ('Best Configuration of Hyper Parameters for Robust Regularization')
Hyper_Parameters[np.argmin(mean_abs_error)] , np.min(mean_abs_error)

Best Configuration of Hyper Parameters for Robust Regularization


(1.0, 1.803755202291363)

In [11]:
model

ElasticNet(alpha=10202.350099442046, copy_X=True, fit_intercept=True,
      l1_ratio=1.0, max_iter=3000, normalize=False, positive=False,
      precompute=False, random_state=0, selection='random', tol=0.0001,
      warm_start=False)